In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Load the dataset
df = pd.read_csv('kaggle/california_housing_prices.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
# Preprocess the dataset
# Drop rows with missing values
df = df.dropna()

# Use one-hot encoding for categorical variables
df = pd.get_dummies(df, drop_first=True)

# Separate features and target
X = df.drop('median_house_value', axis=1).values
y = df['median_house_value'].values

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        # create a neural network with 2 hidden layers
        # the first hidden layer has 128 neurons
        # the second hidden layer has 64 neurons
        # the output layer has 1 neuron
        # use ReLU activation function for hidden layers
        self.fc1 = nn.Linear(input_size, 128) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        # apply ReLU activation function to the first hidden layer
        x = self.relu(self.fc1(x)) 
        # apply ReLU activation function to the second hidden layer
        x = self.relu(self.fc2(x)) 
        # output layer
        x = self.fc3(x) 
        return x

In [5]:
# Convert arrays to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

# Initialize the model, loss function, and optimizer
model = RegressionNN(X_train.shape[1]) # input_size is the number of features
criterion = nn.MSELoss() # Mean Squared Error
optimizer = optim.Adam(model.parameters(), lr=0.01) # Adam is a variant of gradient descent, lr is the learning rate

# Training loop
epochs = 1_000
for epoch in range(epochs+1):
    model.train() # Set the model to training mode
    optimizer.zero_grad() # Reset gradients
    outputs = model(X_train_tensor) # Forward pass
    loss = criterion(outputs, y_train_tensor) # Compute loss
    loss.backward() # Backward pass
    optimizer.step() # Update weights
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch:4d}/{epochs}, Loss: {loss.item():15.0f}')


Epoch    0/1000, Loss:     55938826240
Epoch   10/1000, Loss:     55929446400
Epoch   20/1000, Loss:     55883456512
Epoch   30/1000, Loss:     55746031616
Epoch   40/1000, Loss:     55428071424
Epoch   50/1000, Loss:     54806753280
Epoch   60/1000, Loss:     53734883328
Epoch   70/1000, Loss:     52058660864
Epoch   80/1000, Loss:     49643401216
Epoch   90/1000, Loss:     46408912896
Epoch  100/1000, Loss:     42367705088
Epoch  110/1000, Loss:     37657137152
Epoch  120/1000, Loss:     32546555904
Epoch  130/1000, Loss:     27377459200
Epoch  140/1000, Loss:     22480332800
Epoch  150/1000, Loss:     18149445632
Epoch  160/1000, Loss:     14664834048
Epoch  170/1000, Loss:     12194122752
Epoch  180/1000, Loss:     10669531136
Epoch  190/1000, Loss:      9808983040
Epoch  200/1000, Loss:      9290304512
Epoch  210/1000, Loss:      8910913536
Epoch  220/1000, Loss:      8596624384
Epoch  230/1000, Loss:      8328798720
Epoch  240/1000, Loss:      8097284096
Epoch  250/1000, Loss:   

In [6]:
model.eval() # Set the model to evaluation mode
with torch.no_grad(): # Turn off gradient computation for validation to save time
    predictions = model(X_test_tensor) # Make predictions
    mse = criterion(predictions, y_test_tensor) # Calculate the loss

print(f'Test MSE: {mse.item()}')

Test MSE: 4495433728.0
